## HORNBILL SUITABILITY USING AUTO-ENCODER BASED TRAINING

In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os
import pickle
import matplotlib.pyplot as plt

def create_comprehensive_autoencoder(input_dim):
    """
    Create a comprehensive autoencoder to handle multiple features
    
    Parameters:
    - input_dim: Number of input features
    
    Returns:
    - Compiled autoencoder model
    """
    # Input layer
    input_layer = tf.keras.layers.Input(shape=(input_dim,))
    
    # Encoder with multiple layers and increasing complexity
    encoded = tf.keras.layers.Dense(
        max(input_dim, 64),  # Increased base layer size
        activation='relu', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001)
    )(input_layer)
    
    # Batch normalization for stability
    encoded = tf.keras.layers.BatchNormalization()(encoded)
    
    # Additional hidden layers with decreasing dimensions
    encoded = tf.keras.layers.Dense(
        max(input_dim // 2, 32),  
        activation='relu', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001)
    )(encoded)
    
    # Batch normalization
    encoded = tf.keras.layers.BatchNormalization()(encoded)
    
    # Bottleneck layer
    bottleneck = tf.keras.layers.Dense(
        max(input_dim // 4, 16),  # Tight bottleneck
        activation='relu', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001)
    )(encoded)
    
    # Decoder with symmetric architecture
    decoded = tf.keras.layers.Dense(
        max(input_dim // 2, 32), 
        activation='relu', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001)
    )(bottleneck)
    
    # Batch normalization
    decoded = tf.keras.layers.BatchNormalization()(decoded)
    
    decoded = tf.keras.layers.Dense(
        max(input_dim, 64), 
        activation='relu', 
        kernel_regularizer=tf.keras.regularizers.l2(0.001)
    )(decoded)
    
    # Final reconstruction layer
    decoded = tf.keras.layers.Dense(
        input_dim, 
        activation='linear'
    )(decoded)
    
    # Create autoencoder
    autoencoder = tf.keras.Model(input_layer, decoded)
    
    # Compile with adaptive optimizer
    autoencoder.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=0.001, 
            clipnorm=1.0  # Gradient clipping for stability
        ),
        loss='mean_squared_error'
    )
    
    return autoencoder

def plot_training_history(history):
    """
    Generate and save a plot of training and validation loss
    
    Parameters:
    - history: Model training history
    """
    plt.figure(figsize=(10, 6))
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Autoencoder Model Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (Mean Squared Error)')
    plt.legend()
    plt.tight_layout()
    
    # Ensure models directory exists
    os.makedirs("models", exist_ok=True)
    
    # Save the plot
    plt.savefig('models/training_loss_plot.png')
    plt.close()

def iqr_outlier_removal(data, columns):
    """
    Remove outliers using Interquartile Range (IQR) method
    
    Parameters:
    - data: DataFrame to clean
    - columns: List of columns to apply IQR cleaning
    
    Returns:
    - Cleaned DataFrame
    """
    cleaned_data = data.copy()
    
    for column in columns:
        # Calculate Q1, Q3, and IQR
        Q1 = cleaned_data[column].quantile(0.05)
        Q3 = cleaned_data[column].quantile(0.95)
        IQR = Q3 - Q1
        
        # Define outlier bounds
        lower_bound = Q1 - 0.1 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Remove outliers
        cleaned_data = cleaned_data[
            (cleaned_data[column] >= lower_bound) & 
            (cleaned_data[column] <= upper_bound)
        ]
        
        print(f"\nOutlier Cleaning for {column}:")
        print(f"  Q1: {Q1:.4f}")
        print(f"  Q3: {Q3:.4f}")
        print(f"  IQR: {IQR:.4f}")
        print(f"  Lower Bound: {lower_bound:.4f}")
        print(f"  Upper Bound: {upper_bound:.4f}")
        print(f"  Removed Rows: {len(data) - len(cleaned_data)}")
    
    return cleaned_data

def advanced_data_preprocessing(data):
    """
    Comprehensive data preprocessing pipeline
    
    Parameters:
    - data: Original DataFrame
    
    Returns:
    - Preprocessed DataFrame
    """
    # Select relevant columns
    relevant_columns = [
        'MO','ALLSKY_SFC_SW_DWN', 'T2M', 'T2MDEW', 'T2M_RANGE', 
        'T2M_MAX', 'T2M_MIN', 'QV2M', 'RH2M', 'PRECTOTCORR', 
        'PS', 'WS10M', 'WS10M_MAX', 'WS10M_MIN',
        'WS50M', 'WS50M_MAX', 'WS50M_MIN', 'NDVI', 'CI', 'ELEVATION'
    ]
    
    # Initial data cleaning
    cleaned_data = data[relevant_columns].copy()
    
    # Convert to numeric and handle errors
    cleaned_data = cleaned_data.apply(pd.to_numeric, errors='coerce')
    
    # Remove rows with NaN values
    cleaned_data = cleaned_data.dropna()
    
    # IQR Outlier Removal
    cleaned_data = iqr_outlier_removal(cleaned_data, relevant_columns)
    
    # Additional preprocessing steps
    def additional_cleaning(df):
        # Remove extreme outliers (6 standard deviations)
        for column in df.columns:
            mean = df[column].mean()
            std = df[column].std()
            df = df[
                (df[column] >= mean - 6*std) & 
                (df[column] <= mean + 6*std)
            ]
        return df
    
    cleaned_data = additional_cleaning(cleaned_data)
    
    # Descriptive statistics before and after cleaning
    print("\n--- Data Cleaning Summary ---")
    print(f"Original Data Rows: {len(data)}")
    print(f"Cleaned Data Rows: {len(cleaned_data)}")
    print(f"Rows Removed: {len(data) - len(cleaned_data)}")
    
    # Visualization of data distribution before and after cleaning
    def plot_distribution_comparison(original, cleaned, columns):
        plt.figure(figsize=(20, 15))
        for i, column in enumerate(columns, 1):
            plt.subplot(5, 4, i)
            plt.hist(original[column], bins=50, alpha=0.5, label='Original')
            plt.hist(cleaned[column], bins=50, alpha=0.5, label='Cleaned')
            plt.title(column)
            plt.legend()
        plt.tight_layout()
        plt.savefig('distribution_comparison.png')
        plt.close()
    
    plot_distribution_comparison(data[relevant_columns], cleaned_data, relevant_columns)
    
    return cleaned_data


def train_comprehensive_anomaly_model(data):
    """
    Train a comprehensive anomaly detection model using all relevant features
    
    Parameters:
    - data: Full dataset
    
    Returns:
    - Dictionary with model and scaler
    """
    try:
        # Ensure models directory exists
        os.makedirs("models", exist_ok=True)

        # Select all relevant columns
        relevant_columns = [
            'MO', 'ALLSKY_SFC_SW_DWN', 'T2M', 'T2MDEW', 'T2M_RANGE', 'T2M_MAX', 'T2M_MIN',
            'QV2M', 'RH2M', 'PRECTOTCORR', 'PS', 'WS10M', 'WS10M_MAX', 'WS10M_MIN',
            'WS50M', 'WS50M_MAX', 'WS50M_MIN', 'NDVI', 'CI', 'ELEVATION'
        ]
        
        feature_data = advanced_data_preprocessing(data)
        
        # Convert to numeric and handle errors
        feature_data = feature_data.apply(pd.to_numeric, errors='coerce')
        
        # Skip if insufficient data
        if len(feature_data) < 500:  # Increased minimum samples
            print("Insufficient data for comprehensive model")
            return None
        
        # Scale the data
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(feature_data.values)
        
        # Split data (90% train, 10% test)
        X_train, X_test = train_test_split(X_scaled, test_size=0.1, random_state=42)
        
        # Create and train autoencoder
        input_dim = X_scaled.shape[1]
        autoencoder = create_comprehensive_autoencoder(input_dim)
        
        # Advanced callbacks
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=20, 
            restore_best_weights=True,
            min_delta=0.0001
        )
        
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.5, 
            patience=10, 
            min_lr=0.00001
        )
        
        # Train the model
        history = autoencoder.fit(
            X_train, X_train,
            epochs=250,  # Increased epochs
            batch_size=64,
            shuffle=True,
            validation_data=(X_test, X_test),
            callbacks=[early_stopping, reduce_lr],
            verbose=1
        )

        # Plot training history
        plot_training_history(history.history)

        # Compute reconstruction error for each sample
        train_pred = autoencoder.predict(X_train)
        test_pred = autoencoder.predict(X_test)
        
        train_mse = np.mean(np.power(X_train - train_pred, 2), axis=1)
        test_mse = np.mean(np.power(X_test - test_pred, 2), axis=1)
        
        # Print reconstruction statistics
        print("Training Reconstruction Error:")
        print(f"Mean: {np.mean(train_mse)}")
        print(f"Std: {np.std(train_mse)}")
        print(f"Max: {np.max(train_mse)}")
        
        print("\nTesting Reconstruction Error:")
        print(f"Mean: {np.mean(test_mse)}")
        print(f"Std: {np.std(test_mse)}")
        print(f"Max: {np.max(test_mse)}")

        # Save comprehensive model
        model_path = "models/comprehensive_autoencoder.keras"
        autoencoder.save(model_path)

        # Save scaler using pickle
        scaler_path = "models/comprehensive_scaler.pkl"
        with open(scaler_path, "wb") as f:
            pickle.dump(scaler, f)

        print(f"Saved comprehensive model to {model_path}")
        print (f"Saved scaler to {scaler_path}")
        
        return {
            'autoencoder': autoencoder,
            'scaler': scaler,
            'history': history.history,
            'train_mse': train_mse,
            'test_mse': test_mse
        }
    
    except Exception as e:
        print(f"Error training comprehensive model: {e}")
        return None

def main():
    """
    Train a comprehensive anomaly detection model
    
    Returns:
    - Trained comprehensive model
    """
    # Load your dataset
    df = pd.read_excel("./data/train2.xlsx")
    
    # Remove rows with any empty values
    df = df.dropna()
    
    # Train comprehensive model
    comprehensive_model = train_comprehensive_anomaly_model(df)
    
    return comprehensive_model

if __name__ == "__main__":
    main()


Outlier Cleaning for MO:
  Q1: 1.0000
  Q3: 12.0000
  IQR: 11.0000
  Lower Bound: -0.1000
  Upper Bound: 28.5000
  Removed Rows: 0

Outlier Cleaning for ALLSKY_SFC_SW_DWN:
  Q1: 2.0100
  Q3: 6.3100
  IQR: 4.3000
  Lower Bound: 1.5800
  Upper Bound: 12.7600
  Removed Rows: 1034

Outlier Cleaning for T2M:
  Q1: 13.1100
  Q3: 28.1900
  IQR: 15.0800
  Lower Bound: 11.6020
  Upper Bound: 50.8100
  Removed Rows: 2114

Outlier Cleaning for T2MDEW:
  Q1: 7.6600
  Q3: 24.3900
  IQR: 16.7300
  Lower Bound: 5.9870
  Upper Bound: 49.4850
  Removed Rows: 3203

Outlier Cleaning for T2M_RANGE:
  Q1: 3.9200
  Q3: 13.3500
  IQR: 9.4300
  Lower Bound: 2.9770
  Upper Bound: 27.4950
  Removed Rows: 3849

Outlier Cleaning for T2M_MAX:
  Q1: 19.4900
  Q3: 33.3300
  IQR: 13.8400
  Lower Bound: 18.1060
  Upper Bound: 54.0900
  Removed Rows: 4789

Outlier Cleaning for T2M_MIN:
  Q1: 10.2800
  Q3: 24.8900
  IQR: 14.6100
  Lower Bound: 8.8190
  Upper Bound: 46.8050
  Removed Rows: 5448

Outlier Cleaning for QV2

In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

class FeatureOptimizer:
    def __init__(self, model_path='models/comprehensive_autoencoder.keras', 
                 scaler_path='models/comprehensive_scaler.pkl'):
        """
        Initialize the FeatureOptimizer with pre-trained model and scaler
        
        Parameters:
        - model_path: Path to saved Keras model
        - scaler_path: Path to saved StandardScaler
        """
        # Load the trained model
        self.autoencoder = tf.keras.models.load_model(model_path)
        
        # Load the scaler
        with open(scaler_path, 'rb') as f:
            self.scaler = pickle.load(f)
        
        # Feature names (ensure this matches the order in your original preprocessing)
        self.feature_names = [
            'MO', 'ALLSKY_SFC_SW_DWN', 'T2M', 'T2MDEW', 'T2M_RANGE', 
            'T2M_MAX', 'T2M_MIN', 'QV2M', 'RH2M', 'PRECTOTCORR', 
            'PS', 'WS10M', 'WS10M_MAX', 'WS10M_MIN',
            'WS50M', 'WS50M_MAX', 'WS50M_MIN', 'NDVI', 'CI', 'ELEVATION'
        ]

    def analyze_feature_importance(self, data):
        """
        Analyze the importance of each feature in reconstruction
        
        Parameters:
        - data: Input data DataFrame
        
        Returns:
        - Dictionary of feature importances
        """
        # Preprocess the data
        X_scaled = self.scaler.transform(data[self.feature_names].values)
        
        # Get reconstructed data
        X_reconstructed = self.autoencoder.predict(X_scaled)
        
        # Calculate reconstruction error for each feature
        feature_errors = {}
        for i, feature in enumerate(self.feature_names):
            # Calculate mean squared error for this feature
            feature_mse = np.mean((X_scaled[:, i] - X_reconstructed[:, i])**2)
            feature_errors[feature] = feature_mse
        
        # Sort features by reconstruction error
        sorted_features = sorted(feature_errors.items(), key=lambda x: x[1], reverse=True)
        
        return dict(sorted_features)

    def gradient_feature_importance(self, data):
        """
        Calculate feature importance using gradient-based methods
        
        Parameters:
        - data: Input data DataFrame
        
        Returns:
        - Dictionary of feature importances
        """
        # Preprocess the data
        X_scaled = self.scaler.transform(data[self.feature_names].values)
        
        # Create a model that outputs the reconstruction
        encoder_input = self.autoencoder.layers[0].input
        decoder_output = self.autoencoder.layers[-1].output
        
        # Compute gradients
        feature_importances = {}
        
        with tf.GradientTape() as tape:
            # Convert to tensor and require gradient tracking
            X_tensor = tf.Variable(X_scaled, dtype=tf.float32)
            
            # Predict reconstruction
            reconstructed = self.autoencoder(X_tensor)
            
            # Compute reconstruction loss (Mean Squared Error)
            loss = tf.reduce_mean(tf.square(X_tensor - reconstructed))
        
        # Compute gradients
        gradients = tape.gradient(loss, X_tensor)
        
        # Compute absolute gradient importance
        gradient_importance = np.abs(gradients.numpy()).mean(axis=0)
        
        # Normalize importance
        gradient_importance = gradient_importance / np.sum(gradient_importance)
        
        # Create dictionary of importances
        for feature, importance in zip(self.feature_names, gradient_importance):
            feature_importances[feature] = importance
        
        # Sort by importance
        sorted_importances = dict(sorted(feature_importances.items(), 
                                         key=lambda x: x[1], 
                                         reverse=True))
        
        return sorted_importances

    def permutation_feature_importance(self, data):
        """
        Calculate feature importance using permutation method
        
        Parameters:
        - data: Input data DataFrame
        
        Returns:
        - Dictionary of feature importances
        """
        # Preprocess the data
        X_scaled = self.scaler.transform(data[self.feature_names].values)
        
        # Original reconstruction error
        original_reconstructed = self.autoencoder.predict(X_scaled)
        original_error = np.mean(np.square(X_scaled - original_reconstructed))
        
        # Permutation importance
        feature_importances = {}
        
        for feature_idx, feature in enumerate(self.feature_names):
            # Create a copy of the scaled data
            permuted_X = X_scaled.copy()
            
            # Shuffle the specific feature
            np.random.shuffle(permuted_X[:, feature_idx])
            
            # Reconstruct and compute error
            permuted_reconstructed = self.autoencoder.predict(permuted_X)
            permuted_error = np.mean(np.square(permuted_X - permuted_reconstructed))
            
            # Importance is the increase in reconstruction error
            importance = permuted_error - original_error
            feature_importances[feature] = importance
        
        # Normalize and sort
        total_importance = sum(feature_importances.values())
        normalized_importances = {k: v/total_importance for k, v in feature_importances.items()}
        
        return dict(sorted(normalized_importances.items(), key=lambda x: x[1], reverse=True))

    def generate_comprehensive_report(self, data):
        """
        Generate a comprehensive report of feature analysis
        
        Parameters:
        - data: Input data DataFrame
        """
        # Existing methods
        feature_importance = self.analyze_feature_importance(data)
        
        # New gradient-based importance
        gradient_importance = self.gradient_feature_importance(data)
        permutation_importance = self.permutation_feature_importance(data)
        
        # Create comprehensive report dictionary
        report = {
            'reconstruction_importance': feature_importance,
            'gradient_importance': gradient_importance,
            'permutation_importance': permutation_importance,
            }
        
        return report

# Visualization class remains unchanged
class FeatureImportanceVisualizer:
    def __init__(self, feature_names):
        self.feature_names = feature_names
        self.color_palette = sns.color_palette("husl", len(feature_names))

    def plot_feature_importance_comparison(self, importances):
        plt.figure(figsize=(20, 15))
        plt.subplot(2, 2, 1)
        self._bar_plot(importances['reconstruction_importance'], 
                       title='Reconstruction Error\nFeature Importance')
        plt.subplot(2, 2, 2)
        self._bar_plot(importances['gradient_importance'], 
                       title='Gradient\nFeature Importance')
        plt.subplot(2, 2, 3)
        self._bar_plot(importances['permutation_importance'], 
                       title='Permutation\nFeature Importance')
        plt.subplot(2, 2, 4)
        self._radar_chart(importances)
        plt.tight_layout()
        plt.savefig('comprehensive_feature_importance.png')
        plt.close()

    def _bar_plot(self, importance_dict, title):
        features = list(importance_dict.keys())
        importances = list(importance_dict.values())
        plt.bar(features, importances, color=self.color_palette)
        plt.title(title)
        plt.xticks(rotation=90)
        plt.ylabel('Importance Score')

    def _radar_chart(self, importances):
        normalized_importances = {}
        for method, importance_dict in importances.items():
            if 'importance' in method:
                total = sum(importance_dict.values())
                normalized_importances[method] = {
                    k: v/total for k, v in importance_dict.items()
                }
        methods = list(normalized_importances.keys())
        features = self.feature_names
        angles = np.linspace(0, 2*np.pi, len(features), endpoint=False)
        angles = np.concatenate((angles, [angles[0]]))
        plt.subplot(polar=True)
        for i, method in enumerate(methods):
            values = [normalized_importances[method].get(feature, 0) for feature in features]
            values += [values[0]]
            plt.polar(angles, values, 
                      label=method.replace('_', ' ').title(), 
                      marker='o')
        plt.title('Feature Importance Comparison')
        plt.thetagrids(angles[:-1] * 180/np.pi, features)
        plt.legend(loc='lower right', bbox_to_anchor=(1.2, -0.1))

    def heatmap_feature_correlations(self, data, importances):
        plt.figure(figsize=(15, 12))
        corr_matrix = data[self.feature_names].corr()
        importance_methods = ['reconstruction_importance', 
                              'gradient_importance', 
                              'permutation_importance']
        for i, method in enumerate(importance_methods, 1):
            plt.subplot(1, 3, i)
            sns.heatmap(corr_matrix, 
                        annot=True, 
                        cmap='coolwarm', 
                        center=0,
                        square=True)
            importance_dict = importances[method]
            for y, feature_y in enumerate(self.feature_names):
                for x, feature_x in enumerate(self.feature_names):
                    importance_y = importance_dict.get(feature_y, 0)
                    importance_x = importance_dict.get(feature_x, 0)
                    avg_importance = (importance_y + importance_x) / 2
                    text_color = plt.cm.RdYlGn(avg_importance)
                    plt.gca().texts[y * len(self.feature_names) + x].set_color(text_color)
            plt.title(f'{method.replace("_", " ").title()}\nwith Correlations')
        plt.tight_layout()
        plt.savefig('feature_importance_heatmap.png')
        plt.close()

    def boxplot_feature_distributions(self, data, importances):
        plt.figure(figsize=(20, 10))
        importance_dict = importances['gradient_importance']
        max_importance = max(importance_dict.values())
        for i, feature in enumerate(self.feature_names, 1):
            plt.subplot(4, 5, i)
            sns.boxplot(x=data[feature], 
                        color=plt.cm.YlOrRd(importance_dict[feature]/max_importance))
            plt.title(f'{feature}\nImp: {importance_dict[feature]:.4f}')
            plt.xlabel('')
        plt.tight_layout()
        plt.savefig('feature_distributions.png')
        plt.close()

def main():
    df = pd.read_excel("./data/train.xlsx")
    df = df.dropna()
    optimizer = FeatureOptimizer()
    report = optimizer.generate_comprehensive_report(df)
    visualizer = FeatureImportanceVisualizer(optimizer.feature_names)
    visualizer.plot_feature_importance_comparison(report)
    visualizer.heatmap_feature_correlations(df, report)
    visualizer.boxplot_feature_distributions(df, report)
    
    return report

if __name__ == "__main__":
    main()

3306/3306 ━━━━━━━━━━━━━━━━━━━━ 3s 721us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 3s 809us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 665us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 670us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 665us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 664us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 665us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 667us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 662us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 676us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 668us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 670us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 656us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 684us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 671us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 674us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 663us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 665us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 677us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 676us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 669us/step
3306/3306 ━━━━━━━━━━━━━━━━━━━━ 2s 660us/step


In [34]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

def custom_percentile_rank(values, x):
    """
    Calculate percentile rank for a value or array
    """
    return np.array([np.sum(values <= val) / len(values) * 100 for val in x])

def calculate_similarity(new_place_data, training_data, scaler, weights, autoencoder):
    """
    Calculate similarity between new place data and training data using autoencoder
    
    Parameters:
    - new_place_data: Feature vector of the new place
    - training_data: Scaled feature matrix of training data
    - scaler: Fitted StandardScaler object
    - weights: List of feature weights
    - autoencoder: Trained autoencoder model
    
    Returns:
    - DataFrame with distances and indices of similar places
    """
    # Scale the new place data
    new_place_scaled = scaler.transform(new_place_data.values.reshape(1, -1))
    
    # Apply weights
    training_data_weighted = training_data * weights
    new_place_weighted = new_place_scaled * weights
    
    # Use autoencoder for feature extraction
    training_encoded = autoencoder.predict(training_data_weighted)
    new_place_encoded = autoencoder.predict(new_place_weighted)
    
    # Compute distances (Euclidean and Cosine Similarity) in encoded space
    euclidean_distances = np.linalg.norm(training_encoded - new_place_encoded, axis=1)
    cosine_similarities = cosine_similarity(new_place_encoded, training_encoded).flatten()
    
    # Compute percentile ranks
    euclidean_percentile = custom_percentile_rank(euclidean_distances, euclidean_distances)
    cosine_percentile = custom_percentile_rank(cosine_similarities, cosine_similarities)
    
    # Combine results into a DataFrame
    results = pd.DataFrame({
        'Euclidean_Distance': euclidean_distances,
        'Cosine_Similarity': cosine_similarities,
        'Euclidean_Percentile': euclidean_percentile,
        'Cosine_Percentile': cosine_percentile
    })
    
    return results

def visualize_similarity_distribution(similarity_results, new_place_data):
    """
    Create visualizations of similarity distributions
    """
    plt.figure(figsize=(15, 5))
    
    # Euclidean Distance Distribution
    plt.subplot(1, 3, 1)
    sns.histplot(similarity_results['Euclidean_Distance'], kde=True)
    plt.title('Euclidean Distance Distribution')
    plt.xlabel('Distance')
    plt.ylabel('Frequency')
    plt.axvline(x=similarity_results['Euclidean_Distance'].min(), color='r', linestyle='--', 
                label='Most Similar')
    plt.legend()
    
    # Cosine Similarity Distribution
    plt.subplot(1, 3, 2)
    sns.histplot(similarity_results['Cosine_Similarity'], kde=True)
    plt.title('Cosine Similarity Distribution')
    plt.xlabel('Similarity')
    plt.ylabel('Frequency')
    plt.axvline(x=similarity_results['Cosine_Similarity'].max(), color='r', linestyle='--', 
                label='Most Similar')
    plt.legend()
    
    # Scatter plot of Euclidean vs Cosine
    plt.subplot(1, 3, 3)
    plt.scatter(similarity_results['Euclidean_Distance'], 
                similarity_results['Cosine_Similarity'], 
                alpha=0.5)
    plt.title('Euclidean Distance vs Cosine Similarity')
    plt.xlabel('Euclidean Distance')
    plt.ylabel('Cosine Similarity')
    
    plt.tight_layout()
    plt.savefig('models/similarity_analysis.png')
    plt.close()

def comprehensive_similarity_analysis(new_place_data, training_data, scaler, weights, autoencoder, euclidean_threshold, cosine_threshold):
    """
    Perform comprehensive similarity analysis with dissimilarity check
    """
    # Calculate similarity
    similarity_results = calculate_similarity(
        new_place_data, 
        scaler.transform(training_data.values), 
        scaler, 
        weights,
        autoencoder
    )
    
    # Check if the new place is dissimilar
    min_euclidean_distance = similarity_results['Euclidean_Distance'].min()
    max_cosine_similarity = similarity_results['Cosine_Similarity'].max()
    
    is_dissimilar = (
        min_euclidean_distance > euclidean_threshold or 
        max_cosine_similarity < cosine_threshold
    )
    
    # Find top similar places
    top_similar_euclidean = similarity_results.nsmallest(5, 'Euclidean_Distance')
    top_similar_cosine = similarity_results.nlargest(5, 'Cosine_Similarity')
    
    # Visualize similarity distribution
    visualize_similarity_distribution(similarity_results, new_place_data)
    
    # Detailed analysis
    analysis_results = {
        'new_place_data': new_place_data,
        'most_similar_euclidean': training_data.iloc[top_similar_euclidean.index],
        'most_similar_cosine': training_data.iloc[top_similar_cosine.index],
        'similarity_metrics': similarity_results,
        'is_dissimilar': is_dissimilar
    }
    
    return analysis_results

def main():
    """
    Compare a new place with training places for similarity and dissimilarity
    """
    # Load the scaler
    with open("models/comprehensive_scaler.pkl", "rb") as f:
        scaler = pickle.load(f)
    
    # Load the autoencoder model
    model_path = "models/comprehensive_autoencoder.keras"
    autoencoder = tf.keras.models.load_model(model_path)
    
    # Load your training dataset
    training_data = pd.read_excel("./data/train2.xlsx")
    
    # Filter and scale the training data
    relevant_columns = [
        'MO', 'ALLSKY_SFC_SW_DWN', 'T2M', 'T2MDEW', 'T2M_RANGE', 'T2M_MAX', 'T2M_MIN',
        'QV2M', 'RH2M', 'PRECTOTCORR', 'PS', 'WS10M', 'WS10M_MAX', 'WS10M_MIN',
        'WS50M', 'WS50M_MAX', 'WS50M_MIN', 'NDVI', 'CI', 'ELEVATION'
    ]
    training_features = training_data[relevant_columns].dropna()
    
    # Define weights
    weights = np.ones(len(relevant_columns))
    weights[relevant_columns.index('NDVI')] = 2.7  # Vegetation Density
    weights[relevant_columns.index('CI')] = 2.5    # Choloropyhll Index
    weights[relevant_columns.index('ELEVATION')] = 2.2  # Topographical Influence
    
    # Thresholds for similarity
    euclidean_threshold = 6.5
    cosine_threshold = 0.85
    
    # Load new places dataset
    new_places_data = pd.read_excel("./data/test2.xlsx")
    
    # Select relevant columns, excluding the index and TARGET
    relevant_columns = [col for col in new_places_data.columns if col not in ['TARGET']]
    new_places_data = new_places_data[relevant_columns].dropna()
    
    # Group by LAT and LON and calculate mean for other columns
    grouped_places = new_places_data .groupby(['LAT', 'LON','MO']).agg({
        col: 'mean' for col in new_places_data.columns 
        if col not in ['LAT', 'LON','MO']
    })

    # Reset index to make LAT and LON regular columns again
    grouped_places_reset = grouped_places.reset_index()

    output_results = []

    # Debug: Check the shape of the grouped DataFrame
    print(f"Number of unique LAT,LON combinations: {grouped_places_reset.shape[0]}")

    # Iterate through unique LAT,LON combinations
    for idx, grouped_place_data in grouped_places_reset.iterrows():
        try:
            # Print coordinates
            print(f"\nAnalyzing place for Month{grouped_place_data['MO']} with coordinates (LAT: {grouped_place_data['LAT']}, LON: {grouped_place_data['LON']})")
            
            # Ensure data is in the correct format
            new_place_data_for_analysis = grouped_place_data.drop(['LAT', 'LON','YEAR','DY'])

            # Convert the dictionary values into a pandas DataFrame (single row)
            place_data_df = pd.DataFrame(new_place_data_for_analysis).T
            
            # Calculate similarity and print results
            analysis_results = comprehensive_similarity_analysis(
                place_data_df, 
                training_features, 
                scaler, 
                weights, 
                autoencoder,
                euclidean_threshold, 
                cosine_threshold
            )

            # Store results
            output_results.append({
                'LAT': grouped_place_data['LAT'],
                'LON': grouped_place_data['LON'],
                'Month': grouped_place_data['MO'],
                'Is_Similar': not analysis_results['is_dissimilar'],
                'Cosine_Similarity': analysis_results['similarity_metrics']['Cosine_Similarity'].max(),
                'Euclidean_Distance': analysis_results['similarity_metrics']['Euclidean_Distance'].min()
            })

        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            print(f"Problematic data: {grouped_place_data}")
            continue

        # Save all results to an Excel file
    output_df = pd.DataFrame(output_results)
    output_file = 'models/similarity_analysis_results.xlsx'
    output_df.to_excel(output_file, index=False)
    print(f"All results saved to {output_file}")

if __name__ == "__main__":
    main()

Number of unique LAT,LON combinations: 120

Analyzing place for Month1.0 with coordinates (LAT: -20.8, LON: 31.65)
1576/1576 ━━━━━━━━━━━━━━━━━━━━ 1s 772us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

Analyzing place for Month2.0 with coordinates (LAT: -20.8, LON: 31.65)
1576/1576 ━━━━━━━━━━━━━━━━━━━━ 1s 740us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step

Analyzing place for Month3.0 with coordinates (LAT: -20.8, LON: 31.65)
1576/1576 ━━━━━━━━━━━━━━━━━━━━ 1s 774us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step

Analyzing place for Month4.0 with coordinates (LAT: -20.8, LON: 31.65)
1576/1576 ━━━━━━━━━━━━━━━━━━━━ 1s 780us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

Analyzing place for Month5.0 with coordinates (LAT: -20.8, LON: 31.65)
1576/1576 ━━━━━━━━━━━━━━━━━━━━ 1s 859us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step

Analyzing place for Month6.0 with coordinates (LAT: -20.8, LON: 31.65)
1576/1576 ━━━━━━━━━━━━━━━━━━━━ 1s 774us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

Analyzing place for Month7

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
df = pd.read_excel('./models/similarity_analysis_results.xlsx')

def categorize_similarity(row):
    # Define thresholds for Cosine Similarity and Euclidean Distance
    if row['Cosine_Similarity'] > 0.96 and row['Euclidean_Distance'] <= 1:
        return 'Extremely Suitable for Hornbill'
    elif row['Cosine_Similarity'] > 0.93 and row['Euclidean_Distance'] <= 1.75:
        return 'Ideal Suitable for Hornbill'
    elif row['Cosine_Similarity'] > 0.88 and row['Euclidean_Distance'] <= 2.5:
        return 'Highly Suitable for Hornbill'
    elif row['Cosine_Similarity'] > 0.85 and row['Euclidean_Distance'] < 3.25:
        return 'Moderately Suitable for Hornbill'
    elif row['Cosine_Similarity'] > 0.80 and row['Euclidean_Distance'] < 5:
        return 'Less Suitability for Hornbill'
    elif row['Cosine_Similarity'] > 0.75 and row['Euclidean_Distance'] < 7.5:
        return 'Minimal Suitability for Hornbill'
    else:
        return 'unSuitable for Hornbill'


# Apply categorization to each row
df['Similarity_Category'] = df.apply(categorize_similarity, axis=1)

# Print locations with their similarity class
def print_locations_with_similarity(df):
    # Print the locations (LAT, LON) along with their similarity category
    location_similarity = df[['LAT', 'LON','Month', 'Similarity_Category']]
    location_similarity.to_excel("similarity_category.xlsx",index=False)

# Visualization Function
def visualize_similarity_categories(df):
    # Plot a pie chart for the distribution of similarity categories
    plt.figure(figsize=(8, 8))
    similarity_distribution = df['Similarity_Category'].value_counts()
    similarity_distribution.plot(kind='pie', autopct='%1.1f%%', colors=['#66b3ff', '#99ff99', '#ff6666'])
    plt.title('Similarity Category Distribution')
    plt.tight_layout()
    plt.savefig('similarity_category_distribution_pie.png')
    plt.close()

    # Scatter plot of Cosine Similarity vs Euclidean Distance colored by similarity category
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df, x='Cosine_Similarity', y='Euclidean_Distance', hue='Similarity_Category', palette='coolwarm')
    plt.title('Cosine Similarity vs Euclidean Distance')
    plt.xlabel('Cosine Similarity')
    plt.ylabel('Euclidean Distance')
    plt.tight_layout()
    plt.savefig('cosine_vs_euclidean_similarity.png')
    plt.close()

# Main Execution
def main():
    # Print locations with their similarity class
    print_locations_with_similarity(df)
    
    # Visualize similarity categories
    visualize_similarity_categories(df)

if __name__ == "__main__":
    main()


In [39]:
import pandas as pd
import numpy as np

def combine_datasets(ground_truth_path, similarity_path):
    """
    Combine ground truth and similarity datasets
    
    Parameters:
    -----------
    ground_truth_path : str
        Path to ground truth Excel file
    similarity_path : str
        Path to similarity category Excel file
    
    Returns:
    --------
    pd.DataFrame
        Combined dataset with binary suitability classification
    """
    # Read datasets
    ground_truth = pd.read_excel(ground_truth_path)
    similarity_data = pd.read_excel(similarity_path)
    
    
    # Define positive suitability categories
    positive_categories = [
        'Extremely Suitable for Hornbill',
        'Ideal Suitable for Hornbill',
        'Highly Suitable for Hornbill',
        'Moderately Suitable for Hornbill'
    ]
    
    # Aggregate similarity data by location (if multiple entries)
    grouped_similarity = similarity_data.groupby(['LAT', 'LON']).agg({
        'Similarity_Category': lambda x: x.value_counts().index[0],
        'Month': 'first'  # Keep first month if multiple
    }).reset_index()
    
    # Create binary suitability column
    grouped_similarity['is_suitable'] = grouped_similarity['Similarity_Category'].isin(positive_categories)
    
    # Combine datasets
    combined_results = pd.merge(
        ground_truth, 
        grouped_similarity, 
        on=['LAT', 'LON'], 
        how='outer',
        suffixes=('_truth', '_similarity')
    )
    
    # Fill NaN values
    combined_results['is_suitable'] = combined_results['is_suitable'].fillna(False)
    combined_results['has_hornbill'] = combined_results['has_hornbill'].fillna(False)
    
    # Verification and Analysis
    print("\nDataset Combination Summary:")
    print(f"Total Locations: {len(combined_results)}")
    print(f"Locations with Hornbills: {combined_results['has_hornbill'].sum()}")
    print(f"Locations with Suitable Habitat: {combined_results['is_suitable'].sum()}")
    
    # Confusion Matrix-like Analysis
    true_positive = ((combined_results['has_hornbill'] == True) & (combined_results['is_suitable'] == True)).sum()
    true_negative = ((combined_results['has_hornbill'] == False) & (combined_results['is_suitable'] == False)).sum()
    false_positive = ((combined_results['has_hornbill'] == False) & (combined_results['is_suitable'] == True)).sum()
    false_negative = ((combined_results['has_hornbill'] == True) & (combined_results['is_suitable'] == False)).sum()
    
    print("\nPrediction Analysis:")
    print(f"True Positives: {true_positive}")
    print(f"True Negatives: {true_negative}")
    print(f"False Positives: {false_positive}")
    print(f"False Negatives: {false_negative}")

    combined_results=combined_results[['LAT','LON','Similarity_Category','has_hornbill','is_suitable']]
    
    # Save combined results
    combined_results.to_excel('combined_hornbill_results.xlsx', index=False)
    
    return combined_results

def calculate_metrics(combined_results):
    """
    Calculate performance metrics
    
    Parameters:
    -----------
    combined_results : pd.DataFrame
        Combined dataset with ground truth and predictions
    
    Returns:
    --------
    dict: Performance metrics
    """
    from sklearn.metrics import (
        accuracy_score, 
        precision_score, 
        recall_score, 
        f1_score, 
        confusion_matrix
    )
    
    # Prepare data for metrics
    y_true = combined_results['has_hornbill']
    y_pred = combined_results['is_suitable']
    
    # Calculate metrics
    metrics = {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'F1 Score': f1_score(y_true, y_pred)
    }
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Print metrics
    print("\nPerformance Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")
    
    print("\nConfusion Matrix:")
    print(cm)
    
    return metrics

def main():
    # Paths to input files
    ground_truth_path = 'data/dt_2.xlsx'
    similarity_path = 'similarity_category.xlsx'
    
    # Combine datasets
    combined_results = combine_datasets(
        ground_truth_path, 
        similarity_path
    )
    
    # Calculate performance metrics
    metrics = calculate_metrics(combined_results)

if __name__ == "__main__":
    main()


Dataset Combination Summary:
Total Locations: 10
Locations with Hornbills: 7
Locations with Suitable Habitat: 8

Prediction Analysis:
True Positives: 7
True Negatives: 2
False Positives: 1
False Negatives: 0

Performance Metrics:
Accuracy: 0.9000
Precision: 0.8750
Recall: 1.0000
F1 Score: 0.9333

Confusion Matrix:
[[2 1]
 [0 7]]


In [40]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Create the DataFrame
data = pd.read_excel("combined_hornbill_results.xlsx")

df = pd.DataFrame(data)

# Create a color mapping for suitability categories
color_map = {
    'unSuitable for Hornbill': 'red',
    'Less Suitability for Hornbill': 'orange',
    'Minimal Suitability for Hornbill': 'lightred',
    'Moderately Suitable for Hornbill': 'lightgreen',
    'Highly Suitable for Hornbill': 'green',
    'Extremely Suitable for Hornbill': 'darkgreen',
    'Ideal Suitable for Hornbill': 'darkpurple'
}

# Create a map centered on the mean latitude and longitude
center_lat = df['LAT'].mean()
center_lon = df['LON'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=3)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each location
for idx, row in df.iterrows():
    # Determine popup content
    popup_content = f"""
    Latitude: {row['LAT']}
    Longitude: {row['LON']}
    Suitability: {row['Similarity_Category']}
    Hornbill Present: {row['has_hornbill']}
    Suitable Habitat: {row['is_suitable']}
    """
    
    # Choose color based on suitability category
    marker_color = color_map.get(row['Similarity_Category'], 'blue')
    
    # Add marker
    folium.Marker(
        location=[row['LAT'], row['LON']],
        popup=popup_content,
        icon=folium.Icon(color=marker_color, icon='info-sign')
    ).add_to(marker_cluster)

# Add a legend
legend_html = """
<div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background-color: white; padding: 10px; border: 2px solid grey;">
<h4>Hornbill Habitat Suitability</h4>
"""
for category, color in color_map.items():
    legend_html += f'<p><span style="color:{color};">●</span> {category}</p>'
legend_html += "</div>"
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map
m.save('hornbill_habitat_map.html')

# Additional Analysis
print("\nSuitability Category Distribution:")
print(df['Similarity_Category'].value_counts())

print("\nHornbill Presence Analysis:")
print(df['has_hornbill'].value_counts())

print("\nSuitability Analysis:")
print(df['is_suitable'].value_counts())


Suitability Category Distribution:
Similarity_Category
Highly Suitable for Hornbill     5
Ideal Suitable for Hornbill      3
unSuitable for Hornbill          1
Less Suitability for Hornbill    1
Name: count, dtype: int64

Hornbill Presence Analysis:
has_hornbill
True     7
False    3
Name: count, dtype: int64

Suitability Analysis:
is_suitable
True     8
False    2
Name: count, dtype: int64
